In [27]:
import pandas as pd
import numpy as np
import time

import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [29]:
# 수집할 맥주 목록
beer_list = ['Apostel Brau',
 'Egger Zwickl', 'Egger Marzenbier', 'Holsten Premium Beer', 'Franzisaner Hefe-Weissbier',
 'Egger Radler Grapefruit', 'Barvaria Premium', 'Barvaria 8.6', 'Lapin Kulta IV A',
 'Grolsch Premium Larger', 'Gambrinus Original', 'XXXX Gold', 'Leffe Brown',
 'Lowenbrau Original', 'Asahi Super dray Black ', 'Harbin Beer', "beck's",
 'Hoegaarden Rosee', 'Platinum White Ale', 'Platinum Pale Ale', 'Ambar Especial Larger',
 'Schöfferhofer Grapefruit', 'Volfas Engelman Grünberger Hefeweizen', 'Berliner Kindl Pilsener ',
 'BURGE MEESTER', 'Red Rock', 'Erdinger Dunkel', 'Warsteiner Premium Verum', "Queen's Ale Blonde Type"]

# 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])
# 저장할 데이터 생성 
data = pd.DataFrame(data=[], columns = ['맥주이름'])

In [9]:
print(beer_list)

                                     검색이름
0                            Apostel Brau
1                            Egger Zwickl
2                        Egger Marzenbier
3                    Holsten Premium Beer
4              Franzisaner Hefe-Weissbier
5                 Egger Radler Grapefruit
6                        Barvaria Premium
7                            Barvaria 8.6
8                        Lapin Kulta IV A
9                  Grolsch Premium Larger
10                     Gambrinus Original
11                              XXXX Gold
12                            Leffe Brown
13                     Lowenbrau Original
14                Asahi Super dray Black 
15                            Harbin Beer
16                                 beck's
17                       Hoegaarden Rosee
18                     Platinum White Ale
19                      Platinum Pale Ale
20                  Ambar Especial Larger
21               Schöfferhofer Grapefruit
22  Volfas Engelman Grünberger Hef

In [22]:
def login():
    
    #print('url_open...{0} 맥주 데이터를 수집합니다..'.format(beer))
    
    browser = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
    url = 'https://www.beeradvocate.com/'
    browser.get(url)
    time.sleep(2)
    
    # 로그인 
    browser.find_element(By.CSS_SELECTOR,'#QuickSearchQuery').send_keys('cass')
    browser.find_element(By.CSS_SELECTOR,'#content > div > div > div.uix_contentFix > aside > div > div > div:nth-child(2) > div > form > input.button.primary.Tooltip').click()
    time.sleep(2)
    email = 'kimnh097@naver.com'
    password = 'skagns0019!'
    element = WebDriverWait(browser, 10).until(
                EC.visibility_of_any_elements_located((By.ID, "ctrl_pageLogin_login")))[0]
    element.send_keys(email)

    element_a= WebDriverWait(browser, 10).until(
                EC.visibility_of_any_elements_located((By.ID, "ctrl_pageLogin_password")))[0]
    element_a.send_keys(password)
    browser.find_element(By.CSS_SELECTOR,'#pageLogin > dl.ctrlUnit.submitUnit > dd > input').send_keys(Keys.ENTER)
    browser.find_element(By.CSS_SELECTOR,'#content > div > div > div.uix_contentFix > aside > div > div > div > div > form > input.button.primary.Tooltip').click()
    
    # 맥주 검색
    time.sleep(2)
    element = browser.find_element(By.XPATH,'//*[@id="ba-content"]/form/input[1]')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys('1664 blanc')
    time.sleep(3)
    # 검색
    browser.find_element(By.CSS_SELECTOR,'#ba-content > form > input.primary.button').click()
    time.sleep(2)
    # 제일 상단 리뷰 들어가기
    browser.find_element(By.CSS_SELECTOR,'#ba-content > div:nth-child(3) > div:nth-child(2) > a:nth-child(2)').click()
    time.sleep(2)
    
    # 사진이 포함된 리뷰 가져오기
    reviews = browser.find_elements(By.XPATH, '//*[@id="rating_fullview_container"]')
    for review in reviews:
        #print(review+1, '번째 페이지')
        tmp = []
        # data global 선언
        global data
        time.sleep(10)
        if review.find_elements(By.XPATH,'//*[@id="rating_fullview_container"]//img'):
            tmp.append(review.text)
        tmp = pd.DataFrame(data = tmp, columns=['리뷰'])
        tmp['맥주이름'] = '1664 blanc'
        data = pd.concat([data,tmp])
        # 다음페이지
        try :
            element = browser.find_element(By.XPATH,'//*[@id="ba-content"]/div[9]/span/a[3]')
            browser.execute_script("arguments[0].click();", element)
            time.sleep(10)
        except:
            print('마지막 페이지입니다.')
    
#     # 데이터를 csv, excel 파일로 저장합니다.
#     result = pd.merge(data, beer_list, on='검색이름', how='left')
#     result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')

#     driver.quit()

#     return result

# # 맥주 리스트 개수만큼 데이터 수집
# for k in range(len(beer_list)):
#     result = crawl(driver, beer_list['검색이름'].iloc[k], data, k)

In [30]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

data = pd.DataFrame(columns=['리뷰', '맥주이름'])

def login():
    
    #print('url_open...{0} 맥주 데이터를 수집합니다..'.format(beer))
    
    browser = webdriver.Chrome(ChromeDriverManager().install())
    url = 'https://www.beeradvocate.com/'
    browser.get(url)
    time.sleep(2)
    
    # 로그인 
    browser.find_element(By.CSS_SELECTOR,'#QuickSearchQuery').send_keys('cass')
    browser.find_element(By.CSS_SELECTOR,'#content > div > div > div.uix_contentFix > aside > div > div > div:nth-child(2) > div > form > input.button.primary.Tooltip').click()
    time.sleep(2)
    email = 'kimnh097@naver.com'
    password = 'skagns0019!'
    element = WebDriverWait(browser, 10).until(
                EC.visibility_of_any_elements_located((By.ID, "ctrl_pageLogin_login")))[0]
    element.send_keys(email)

    element_a= WebDriverWait(browser, 10).until(
                EC.visibility_of_any_elements_located((By.ID, "ctrl_pageLogin_password")))[0]
    element_a.send_keys(password)
    browser.find_element(By.CSS_SELECTOR,'#pageLogin > dl.ctrlUnit.submitUnit > dd > input').send_keys(Keys.ENTER)
    browser.find_element(By.CSS_SELECTOR,'#content > div > div > div.uix_contentFix > aside > div > div > div > div > form > input.button.primary.Tooltip').click()
    
    # 맥주 검색
    time.sleep(2)
    element = browser.find_element(By.XPATH,'//*[@id="ba-content"]/form/input[1]')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys('1664 blanc')
    time.sleep(3)
    # 검색
    browser.find_element(By.CSS_SELECTOR,'#ba-content > form > input.primary.button').click()
    time.sleep(2)
    # 제일 상단 리뷰 들어가기
    browser.find_element(By.CSS_SELECTOR,'#ba-content > div:nth-child(3) > div:nth-child(2) > a:nth-child(2)').click()
    time.sleep(2)
    
    # 사진이 포함된 리뷰 가져오기
    reviews = browser.find_elements(By.XPATH, '//*[@id="rating_fullview_container"]')
    for review in reviews:
        #print(review+1, '번째 페이지')
        tmp = []
        # data global 선언
        data = pd.DataFrame(data=[], columns = ['맥주이름'])
        time.sleep(10)
        if review.find_elements(By.XPATH,'//*[@id="rating_fullview_user"]/div/a/img'):
            tmp.append(review.text)
            tmp = pd.DataFrame(data = tmp, columns=['리뷰'])
            tmp['맥주이름'] = '1664 blanc'
            data = pd.concat([data,tmp])
        # 다음페이지
        while True:
            try:
                # 첫 번째 버튼 클릭
                wait = WebDriverWait(browser, 10)
                element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ba-content"]/div[9]/span/a[3]')))
                element.click()
                time.sleep(10)
            except NoSuchElementException:
                try:
                    # 두 번째 버튼 클릭
                    element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ba-content"]/div[9]/span/a[7]')))
                    element.click()
                    time.sleep(10)
                except NoSuchElementException:
                    # 버튼이 없으면 마지막 페이지임
                    print("마지막 페이지입니다.")
                    break









#             try :
#                 element = browser.find_element(By.XPATH,'//*[@id="ba-content"]/div[9]/span/a[3]')
#                 element1 = browser.find_element(By.XPATH,'//*[@id="ba-content"]/div[9]/span/a[7]')    
#                 if element:
#                     element.click()
#                 elif element1:
#                     element1.click()
#                 #browser.execute_script("arguments[0].click();", element)
#                 time.sleep(10)
#             except NoSuchElementException:
#                 print('마지막 페이지입니다.')
#                 continue

    # 데이터를 csv, excel 파일로 저장합니다.
#     result = pd.merge(data, beer_list,


In [ ]:
login()

In [17]:
!pip3 install webdriver-manager


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip3 install --up

In [47]:
data

,맥주이름,리뷰
0,1664 blanc,Rated by stvo\n\n4.77/5 rDev +32.5%\nlook: 5 ...
0,1664 blanc,Rated by Spark420\n\n2.93/5 rDev -18.6%\nlook...
0,1664 blanc,Reviewed by flyingpig from Scotland\n\n3.17/5 ...
0,1664 blanc,Reviewed by LampertLand from Canada (BC)\n\n3....
0,1664 blanc,Reviewed by dcscientist from Colorado\n\n4/5 ...
0,1664 blanc,Reviewed by Ozzylizard from Pennsylvania\n\n3....
0,1664 blanc,Reviewed by Tony787 from Canada (ON)\n\n3.43/5...
0,1664 blanc,Reviewed by Sigmund from Norway\n\n3/5 rDev -...
0,1664 blanc,Reviewed by StonedTrippin from Colorado\n\n4.0...


In [13]:
for k in range(len(beer_list)):
    result = crawl(browser, beer_list['검색이름'].iloc[k],data,k)

/var/folders/cy/tdjg6lxs4b1cnkfjsnfcfckm0000gp/T/ipykernel_13023/2780140249.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('./chromedriver')
